In [1]:
open Core

Findlib has been successfully loaded. Additional directives:
  #require "package";;      to load a package
  #list;;                   to list the available packages
  #camlp4o;;                to load camlp4 (standard syntax)
  #camlp4r;;                to load camlp4 (revised syntax)
  #predicates "p,q,...";;   to set these predicates
  Topfind.reset();;         to force that packages will be reloaded
  #thread;;                 to enable threads



/home/ceclinux/.opam/4.04.1/lib/ocaml/dynlink.cma: loaded
/home/ceclinux/.opam/4.04.1/lib/ocaml/camlp4: added to search path
/home/ceclinux/.opam/4.04.1/lib/ocaml/camlp4/camlp4o.cma: loaded
/home/ceclinux/.opam/4.04.1/lib/ocaml/threads: added to search path


Cannot find file /home/ceclinux/.opam/4.04.1/lib/ocaml/camlp4/camlp4o.cma.


/home/ceclinux/.opam/4.04.1/lib/ocaml/unix.cma: loaded
/home/ceclinux/.opam/4.04.1/lib/ocaml/threads/threads.cma: loaded
/home/ceclinux/.opam/4.04.1/lib/base/caml: added to search path
/home/ceclinux/.opam/4.04.1/lib/base/caml/caml.cma: loaded
/home/ceclinux/.opam/4.04.1/lib/base/shadow_stdlib: added to search path
/home/ceclinux/.opam/4.04.1/lib/base/shadow_stdlib/shadow_stdlib.cma: loaded
/home/ceclinux/.opam/4.04.1/lib/sexplib/0: added to search path
/home/ceclinux/.opam/4.04.1/lib/sexplib/0/sexplib0.cma: loaded
/home/ceclinux/.opam/4.04.1/lib/base: added to search path
/home/ceclinux/.opam/4.04.1/lib/base/base.cma: loaded
/home/ceclinux/.opam/4.04.1/lib/base/md5: added to search path
/home/ceclinux/.opam/4.04.1/lib/base/md5/md5_lib.cma: loaded
/home/ceclinux/.opam/4.04.1/lib/ocaml/bigarray.cma: loaded
/home/ceclinux/.opam/4.04.1/lib/fieldslib: added to search path
/home/ceclinux/.opam/4.04.1/lib/fieldslib/fieldslib.cma: loaded
/home/ceclinux/.opam/4.04.1/lib/ppx_compare/runtime-lib

Ocaml最棒的特性之一是，它提供了一个简洁而且表达能力很强的系统来声明新的数据类型，记录则是这个系统中一个关键的要素

需要说明，记录字段名必须以一个小写的子母开头

In [2]:
type host_info = 
{hostname: string;
os_name:string;
cpu_arch:string;
timestamp:Time.t;
}

type host_info = {
  hostname : string;
  os_name : string;
  cpu_arch : string;
  timestamp : Core.Time.t;
}


In [3]:
# require "core_extended" 

No such package: core_extended


In [4]:
open Core_extended


error: compile_error

In [5]:
let my_host = 
{
hostname = "hostname";
os_name = "uname -s";
cpu_arch = "uname -p";
timestamp = Time.now();
}

val my_host : host_info =
  {hostname = "hostname"; os_name = "uname -s"; cpu_arch = "uname -p";
   timestamp = 2018-02-01 06:13:47.986132-08:00}


你可能想知道编译器如何推断出`my_host`类型是`host_info`。在这里，编译器确定类型的关键是记录字段名。在这一张后面，我们还会讨论多个记录类型有相同的字段名时会发生什么

可以使用点击法来抽取元素

In [6]:
my_host.cpu_arch;;

- : string = "uname -p"


声明Ocam类型时，都可以利用多态类型来进行参数化，在这方面记录也不例外。

In [7]:
type 'a timestamped = {item: 'a; time: Time.t};;

type 'a timestamped = { item : 'a; time : Core.Time.t; }


In [8]:
let first_timestamped list =
List.reduce list ~f:(fun a b -> if a .time < b.time then a else b)

val first_timestamped : 'a timestamped Core.List.t -> 'a timestamped option =
  <fun>


In [9]:
let host_info_to_string {hostname = h; os_name = os; cpu_arch = c; timestamp = ts;}
= sprintf "%s (%s / %s, on %s)" h os c "wa";;

val host_info_to_string : host_info -> string = <fun>


In [10]:
host_info_to_string my_host

- : string = "hostname (uname -s / uname -p, on wa)"


需要说明的是，我们使用的模式只有一种情况，这里没有使用由|分割的多个情况。我们只要一个模式，因为记录模式是“不可否定的”，只标明运行时记录模式匹配绝对不会失败。这是有道理的，因为记录中的字段集总是一样的。一般来讲，对于有固定结构的类型（如记录和元祖），其模式都是不可否定的，这与有可变结构的类型不同。

举个例子，假设我们希望向`host_info`记录添加一个新字段，名为`os_release`

In [11]:
type host_info = {
hostname: string;
os_name: string;
cpu_arch :string;
os_release: string;
timestamp: Time.t;
}

type host_info = {
  hostname : string;
  os_name : string;
  cpu_arch : string;
  os_release : string;
  timestamp : Core.Time.t;
}


In [12]:
host_info_to_string
let host_info_to_string {hostname = h; os_name = os; cpu_arch = c; timestamp = ts;}
= sprintf "%s (%s / %s, on %s)" h os c "wa";;

- : host_info -> string = <fun>


val host_info_to_string : host_info -> string = <fun>


host_info_to_string的代码不做修改仍能编译。对于这个情况，很显然你可能希望`host_info_to_string`来包含`os_release`，如果类型系统能对这个修改提供一个警告就好了

In [13]:
#warnings "+9"

变量名与记录名恰好相同时，Ocaml提供了一些方便的语法快捷方式，例如，以下函数中的模式会把当前所有地段绑定到同名的变量，这称为字段双关

In [14]:
let host_info_to_string {hostname = h; os_name = os; cpu_arch = c; timestamp = ts;}
= sprintf "%s (%s / %s, on %s)" h os c "wa";;

File "[14]", line 1, characters 24-83:
Warning 9: the following labels are not bound in this record pattern:
os_release
Either bind these labels explicitly or add '; _' to the pattern.


val host_info_to_string : host_info -> string = <fun>


可以禁用对一个给定模式的警告，明确地确认我们打算忽略额外的字段。为此可以为模式增加一个下划线

In [15]:
host_info_to_string
let host_info_to_string {hostname = h; os_name = os; cpu_arch = c; timestamp = ts;_}
= sprintf "%s (%s / %s, on %s)" h os c "wa";;

- : host_info -> string = <fun>


val host_info_to_string : host_info -> string = <fun>


In [16]:
let host_info_to_string {hostname; os_name; cpu_arch; timestamp;_} =
sprintf "%s (%s / %s) <%s>" hostname os_name cpu_arch (Time.to_string timestamp)

val host_info_to_string : host_info -> string = <fun>


字段双管也可以用来构造一个记录。考虑以下的代码，这用来生成host_info记录

In [17]:
let my_host = 
let sh cmd = Shell.sh_one_exn cmd in
let hostname = sh 'hostname' in
let host_name = sh 'uname -s' in
let cpu_arch = sh 'uname -p' in
let os_release = sh 'uname -r' in
{hostname;os_name; cpu_arch; os_release; timestamp}

error: compile_error

编写函数由标签参数构建一个记录时，可以同时利用字段双关和标签双关

In [18]:
let create_host_info ~hostname ~os_name ~cpu_arch ~os_release = 
{os_name; cpu_arch; os_release; hostname = String.lowercase hostname; timestamp = Time.now()};;

val create_host_info :
  hostname:Core.String.t ->
  os_name:string -> cpu_arch:string -> os_release:string -> host_info = <fun>


In [19]:
type log_entry = 
{ session_id: string;
time: Time.t;
important: bool;
message:string;
};;
type heartbeat = 
{
session_id:string;
time: Time.t;
status_message: string;
};;
type logon = 
{
session_id: string;
time: Time.t;
user: string;
credentials: string;
}

type log_entry = {
  session_id : string;
  time : Core.Time.t;
  important : bool;
  message : string;
}


type heartbeat = {
  session_id : string;
  time : Core.Time.t;
  status_message : string;
}


type logon = {
  session_id : string;
  time : Core.Time.t;
  user : string;
  credentials : string;
}


In [20]:
let get_session_id t = t.session_id

val get_session_id : logon -> string = <fun>


在这种情况，`Ocaml`只是选择这个记录字段的最新定义

In [21]:
let get_heartbeat_session_id (t:heartbeat) = t.session_id

val get_heartbeat_session_id : heartbeat -> string = <fun>


In [22]:
let status_and_session t = (t.status_message, t.session_id)

val status_and_session : heartbeat -> string * string = <fun>


In [23]:
let status_and_session t = (t.session_id,t.status_message )

error: compile_error

为什么第一个定义没有类型标注也能成功？而第二个定义没有类型标注就失败了呢？区别在于，第一种情况下，类型检查代码首先考虑`status_message`字段，因此可以得出结论这个记录是一个`heartbeat`。交换顺序后，则会先考虑`session_id`字段，所以会认为这个类型是一个`logon`，这样一来，`t.status_message`就没有意义了。

可以使用不重叠的字段名避免这种二义性，或者更一般的，为每个类型分别构造一个模块。把类型包装在各自的模块中（也是Core中大量使用的方法）

In [32]:
module Log_entry = struct
type t = 
{
session_id: string;
time: Time.t;
important: bool;
message: string;
}
end
module Heartbeat = struct
type t = 
{
session_id: string;
time: Time.t;
user:string;
credentials: string;
}
end

module Log_entry :
  sig
    type t = {
      session_id : string;
      time : Core.Time.t;
      important : bool;
      message : string;
    }
  end


module Heartbeat :
  sig
    type t = {
      session_id : string;
      time : Core.Time.t;
      user : string;
      credentials : string;
    }
  end


你会发现进场遇到遮掩过一种情况：可能需要创建一个新纪录，它与一个已有的记录基本上相同，只是一小部分字段不同。

In [25]:
type client_info = 
{
addr: Unix.Inet_addr.t;
port: int;
user:string;
credentials: string;
last_heartbeat_time: Time.t;
}

type client_info = {
  addr : Core.Unix.Inet_addr.t;
  port : int;
  user : string;
  credentials : string;
  last_heartbeat_time : Core.Time.t;
}


Modules in OCaml are implemented by module definitions that have the following syntax:

module ModuleName = struct 
  (* definitions *)
end

In [28]:
module ListStack = struct
  let empty = []
  let is_empty s = (s = [])

  let push x s = x :: s

  let peek = function
    | [] -> failwith "Empty"
    | x::_ -> x

  let pop = function
    | [] -> failwith "Empty"
    | _::xs -> xs
end

module ListStack :
  sig
    val empty : 'a list
    val is_empty : 'a list -> bool
    val push : 'a -> 'a list -> 'a list
    val peek : 'a list -> 'a
    val pop : 'a list -> 'a list
  end


is called a structure. A structure is simply a sequence of definitions. The structure itself is anonymous—it has no name—until it is bound to a name by a module definition.

Modules partition the namespace, so that any symbol x that is bound in the implementation of a module named Module must be referenced by the qualifed name Module.x outside the implementation of the module (unless the namespace has been exposed using open).

The implementation of a module can contain type definitions, exception definitions, let definitions, open statements, as well as some other things we haven't seen so far. All the definitions inside a module are permitted to end with double semicolon ;; for compatibility with the toplevel, but 3110 considers it unidiomatic to do so.

Modules are not as first-class in OCaml as functions. There are some language extensions that make it possible to bundle up modules as values, but we won't be looking at them. If you're curious you can have a look at the manual.

In [34]:
let register_heartbeat t hb = 
{
addr = t.addr;
port = t.port;
user = t.user;
credentials = t.credentials;
last_heartbeat_time = hb.Heartbeat.time;
}

val register_heartbeat : client_info -> Heartbeat.t -> client_info = <fun>


In [35]:
let create_log_entry ~session_id ~important message = 
{
Log_entry.time = Time.now(); Log_entry.session_id;
Log_entry.important; Log_entry.message;
}

val create_log_entry :
  session_id:string -> important:bool -> string -> Log_entry.t = <fun>


模块名Log_entry是必要的，用于限定字段，因为这个函数的在定义记录的`Log_entry`模块之外。不过，`Ocaml`只要求对一个记录字段用模块限定，所以这个代码还可以写的更简洁。需要说明，允许在模块路径和字段名之间插入空白符

这种技巧不仅限于构造记录，完成模式匹配时也可以使用相同的技巧

In [37]:
let message_to_string {Log_entry.important; message;_} = if important then String.uppercase message else message;;

val message_to_string : Log_entry.t -> Core.String.t = <fun>


In [41]:
let is_important t = t.Log_entry.important

val is_important : Log_entry.t -> bool = <fun>


**要记住，点（.)有两种用法**

第一个点是访问一个记录字段，点有比那的所有部分都要解释为一个字段名；第二个点是访问一个模块的内容，只是模块`Log_entry`中的记录字段`important`。`Log_entry`首字母大写，所以这不可能是一个字段名，可以用这一点区分这两种用法

与大多数`Ocaml`值类似，记录默认是不可变的。不过，可以把单个的记录字段声明为可变字段。

In [50]:
type client_info = {
addr:Unix.Inet_addr.t;
port:int;
user:string;
credentials:string;
mutable last_heartbeat_time: Time.t;
mutable last_hreatbeat_status:string;
}

type client_info = {
  addr : Core.Unix.Inet_addr.t;
  port : int;
  user : string;
  credentials : string;
  mutable last_heartbeat_time : Core.Time.t;
  mutable last_hreatbeat_status : string;
}


`<-`操作符用于设置可变字段。相应的`register_heartbeat`版本可以写为：

In [55]:
let register_heartbeat t hb = 
t.last_heartbeat_time <- hb.Heartbeat.time;


val register_heartbeat : client_info -> Heartbeat.t -> unit = <fun>
